# Project Air Quality
You will be asked to implement several functions. Team work is not allowed. Everybody implements his/her own code. Discussing issues with others is fine, sharing code with others is not. 

### Dataset: Air quality
The dataset contains 9358 instances of hourly averaged responses from an array of 5 metal oxide chemical sensors embedded in an Air Quality Chemical Multisensor Device. The device was located on the field in a significantly polluted area, at road level, within an Italian city. Data were recorded from March 2004 to February 2005 (one year) representing the longest freely available recordings of on field deployed air quality chemical sensor devices responses. Ground Truth hourly averaged concentrations for CO, Non Metanic Hydrocarbons, Benzene, Total Nitrogen Oxides (NOx) and Nitrogen Dioxide (NO2) and were provided by a co-located reference certified analyzer [1].

**Attributes of the dataset are:**

|Sl No|Attribute|Description|
|-|-|-|
|0|Date|Date (DD/MM/YYYY) |
|1|Time|Time (HH.MM.SS) |
|2|CO(GT)|True hourly averaged concentration CO in mg/m^3 (reference analyzer) |
|3|PT08.S1(CO)|PT08.S1 (tin oxide) hourly averaged sensor response (nominally CO targeted)|
|4|NMHC(GT)|True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3 (reference analyzer)|
|5|C6H6(GT)|True hourly averaged Benzene concentration in microg/m^3 (reference analyzer) |
|6|PT08.S2(NMHC)|PT08.S2 (titania) hourly averaged sensor response (nominally NMHC targeted) |
|7|NOx(GT)|True hourly averaged NOx concentration in ppb (reference analyzer) |
|8|PT08.S3(NOx)|PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally NOx targeted) |
|9|NO2(GT)|True hourly averaged NO2 concentration in microg/m^3 (reference analyzer) |
|10|PT08.S4(NO2)|PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted) |
|11|PT08.S5(O3)|PT08.S5 (indium oxide) hourly averaged sensor response (nominally O3 targeted) |
|12|T|Temperature in Â°C |
|13|RH|Relative Humidity (%) |
|14|AH|AH Absolute Humidity|


### Problem:
Humans are very sensitive to humidity, as the skin relies on the air to get rid of moisture. The process of sweating is your body's attempt to keep cool and maintain its current temperature. If the air is at 100-percent relative humidity, sweat will not evaporate into the air. As a result, we feel much hotter than the actual temperature when the relative humidity is high. If the relative humidity is low, we can feel much cooler than the actual temperature because our sweat evaporates easily, cooling us off. 
The humidity of the air, if it is not maintained at optimal levels, can be a factor that has adverse affects on people's health. According to reports, the human body is said to be most comfortable when the relative humidity of the area ranges between 20 and 60%.


### Objective:
So we will **predict the Relative Humidity** of a given point of time based on the all other attributes affecting the change in RH.


### References:
[1] S. De Vito, E. Massera, M. Piga, L. Martinotto, G. Di Francia, On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario, Sensors and Actuators B: Chemical, Volume 129, Issue 2, 22 February 2008, Pages 750-757, ISSN 0925-4005


### <u>Content:<u>

[1) Load data ](#load_data)
    
[2) Basic statistics](#stat)

[3) Data Cleaning](#hr)
    
[4) Co-relation between variables](#corr)

[5) Influence of features on output-RH](#lin)

[6) Baseline Linear Regression](#LR)

[7) Feature Engineering and testing model](#FE)

[8) Decision Tree Regression ](#DT)
    
[9) Random Forest Regression](#RF) 
    
[9.1) Blox plot](#bxplot)

[11) Conclusion](#conc)


In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns
rcParams['figure.figsize']=10,8

#### 1) Load data<a name="load_data"></a>
We will load the data and have a basic first look at the data.

- Use `pandas.read_csv('path/to/dataset.csv',header=None,skiprows=1,names=col,na_filter=True, na_values=-200,usecols=use)` to load the data. 

In [2]:
# Defining header
col=['DATE','TIME','CO_GT','PT08_S1_CO','NMHC_GT','C6H6_GT','PT08_S2_NMHC',
     'NOX_GT','PT08_S3_NOX','NO2_GT','PT08_S4_NO2','PT08_S5_O3','T','RH','AH']

# Defining number of columns from csv
use = list(np.arange(len(col)))

# Reading the data from csv
df_air = pd.read_csv(
    'data/AirQualityUCI.csv',  # Write the path of the Air Quality Dataset
    header=None, skiprows=1, names=col, na_filter=True, na_values=-200, usecols=use
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/AirQualityUCI.csv'

- Visualize the first and the last 7 rows of the data.

In [ ]:
df_air.head(7)

In [ ]:
df_air.tail(7)

- Print the dtypes in the dataframe

In [ ]:
df_air.info()

- Print dataframe shape

In [ ]:
df_air.shape

#### 2) Basic statistics<a name="stat"></a>
Here we look at basic statistics. This is always helpful to get a feel for your data and might help to find some problems in your data.
- Print dataframe statistics (mean, max & min values for each column etc.)

In [ ]:
df_air.describe()

#### 3) Data Cleaning<a name="hr"></a>
Here we clean up some missing values.

- Drop **ONLY** rows containing **ALL** NaN (not a number) values 
- NOTE: You can use `inplace=True` (See: https://www.geeksforgeeks.org/what-does-inplace-mean-in-pandas/)

In [ ]:
df_air.dropna(how='all', axis=0, inplace=True)

- Now, drop **ONLY** rows with the `thresh=10` NaN values

In [ ]:
df_air.dropna(thresh=10, axis=0, inplace=True)

- How many missing values are in the dataset? Print the number.

In [ ]:
print('Count of missing values:\n')
# Count the invalid values
df_air.isna().sum().sum()

##### Fill missing value strategy
1. CO_GT, NOX_GT, NO2_GT will be filled by monthly average of that particular hour
2. NHHC_GT will be dropped as it has 90% missing data

In [ ]:
df_air['DATE'] = pd.to_datetime(df_air.DATE, format='%m/%d/%Y')   #Format date column (See: https://docs.python.org/3/library/datetime.html)

In [ ]:
# Creating "MONTH" column
df_air['MONTH']=df_air.index.month     
df_air.reset_index(inplace=True) # Run this line only once!

In [ ]:
# Splitting hour from time into new column (See: https://docs.python.org/3/library/stdtypes.html#string-methods)
df_air['HOUR'] = df_air['TIME'].apply(lambda x: int(x.split(':')[0]))
df_air.HOUR.head()

- set DATE as the index 

In [ ]:
df_air.set_index('DATE', inplace=True)

- Drop column NMHC_GT; it has 90% missing data

In [ ]:
df_air.drop(columns='NMHC_GT', inplace=True)

- Fill NaN values with monthly average of particular hour  (See: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) HINT: use `.groupby(['MONTH','HOUR'])`

In [ ]:
df_air['CO_GT']=df_air['CO_GT'].fillna(df_air.groupby(['MONTH','HOUR'])['CO_GT'].transform('mean'))
df_air['NOX_GT']=df_air['NOX_GT'].fillna(df_air.groupby(['MONTH','HOUR'])['NOX_GT'].transform('mean'))
df_air['NO2_GT']=df_air['NO2_GT'].fillna(df_air.groupby(['MONTH','HOUR'])['NO2_GT'].transform('mean'))

- Print missing values per attribute

In [ ]:
print('Left out missing value:')
# Count them
df_air.shape[0] - df_air.count()

 - Fill NaN values with hourly average value HINT: use `.groupby(['HOUR'])`

In [ ]:
# Filling left out NaN values with hourly average value
df_air['CO_GT'] = df_air['CO_GT'].fillna(df_air.groupby(['HOUR'])['CO_GT'].transform('mean'))
df_air['NOX_GT'] = df_air['NOX_GT'].fillna(df_air.groupby(['HOUR'])['NOX_GT'].transform('mean'))
df_air['NO2_GT'] = df_air['NO2_GT'].fillna(df_air.groupby(['HOUR'])['NO2_GT'].transform('mean'))

#### 4) Understand correlation between variables<a name="corr"></a>

 - Display a heatmap using `sns.heatmap` to see correlation between variables

In [ ]:
sns.heatmap(df_air.select_dtypes(include="number").corr(), annot=True, cmap='BuGn', fmt=".2f")
plt.show()

 - Describe the heatmap using your own words

In [ ]:
# ANSWER: ...

#### 5) Try to understand degree of linearity between RH output and other input features<a name="lin"></a>

 - plot all features (x-axis) against output variable RH (y-axis) using `sns.lmplot`. 
 - describe the results

In [ ]:
df_air.head()

In [ ]:
# Ensure numeric columns are properly converted
df_air = df_air.apply(pd.to_numeric, errors='ignore')

# Exclude non-numeric columns from plotting
numeric_columns = df_air.select_dtypes(include="number").columns


# Plot all numeric features against output variable RH
fig, axes = plt.subplots(2, 8, figsize=(20, 7))
for i, feature in enumerate(numeric_columns):
    sns.regplot(data=df_air, x=feature, y='RH', ax=axes[i // 8, i % 8], line_kws=dict(color="#4ea373"), color="gray", marker='.')



In [ ]:
#ANSWER: ...

### 6) Linear Regression<a name="LR"></a>

In [ ]:
from sklearn.preprocessing import StandardScaler         #import normalisation package
from sklearn.model_selection import train_test_split      #import train test split
from sklearn.linear_model import LinearRegression         #import linear regression package
from sklearn.metrics import mean_squared_error,mean_absolute_error   #import mean squared error and mean absolute error

- Define Feature (as X) and Target (as y)

In [ ]:
df_air.keys()

In [ ]:
X = df_air.drop(columns=['DATE', 'TIME', 'RH'])  # X-input features
y = df_air['RH']  # y-input features

- Plot distribution of target variable.

In [ ]:
sns.displot(data=df_air, x=y)
plt.show()

##### Train test split:
 - split the data into train (70%) and test(30%), use a fixed random seed
 - print the size of the train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print('Training data size:')
print(X_train.shape)

print('Test data size:')
print(X_test.shape)

- Normalize data using `StandardScaler`
    - Be careful about which data you use to fit the normalization.
    - apply the same normalisation to the test data as to the train data
    - DO NOT forget to use the normalization for each model (SVR etc.)

In [ ]:
normalizer = StandardScaler()

X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

 - Train the Linear Regression model (See: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

- Print intercept and slopes

In [ ]:
print('Intercept: \t\t\t\t{}'.format(model_lr.intercept_))
for slope, key in zip(model_lr.coef_, X.keys()):
    print('Slope of {:20}: \t\t{}'.format(key, slope))

- Predict on the test data
- Compute and print performance metrics as RMSE. This will be our baseline!

In [ ]:
print('Baseline RMSE:')
y_pred = model_lr.predict(X_test)
baseline_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
baseline_rmse

#### <u>6a) Conclusion of baseline linear regression model:<a name="LRcon"></a>

In [ ]:
# write your conclusion here

### 7) Feature engineering and testing model:<a name="FE"></a>

We will try the model with multiple feature combination and see if RMSE is improving

- Write function to measure RMSE with different combinations of features (try at least 3 combinations of your choice)

In [ ]:
def train_test_RMSE(df_air, feat_):    
    """
    The function train_test_RMSE returns the RMSE for different combinations 
    of features feat_ of the dataframe df_air.
    
        :param df_air: (pandas.DataFrame) Our dataset
        :param feat_: (List[str]) A list of column names
        :return: (float) The score value
    """
    X = df_air[feat_]
    y = df_air['RH']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    normalizer = StandardScaler()
    X_train = normalizer.fit_transform(X_train)
    X_test = normalizer.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# Trail 1
train_test_RMSE(df_air, X.keys()[3:9])

In [ ]:
# Trail 2
train_test_RMSE(df_air, X.keys()[5:11])

In [ ]:
# Trail 3
train_test_RMSE(df_air, X.keys()[3:])

#### <u>7a) Conclusion of Feature Engineering and testing:<a name="FEcon"></a>

In [ ]:
# write your conclusion here

### 8) Decision Tree Regression<a name="DT"></a>

Let us try to apply Decision tree regression technique and see if any improvement happens

In [ ]:
from sklearn.tree import DecisionTreeRegressor         # Decision tree regression model


- Fit the DT model and predict:

In [ ]:
model_dt = DecisionTreeRegressor()
model_dt.fit(X_train, y_train)
y_pred = model_dt.predict(X_test)

- calculate the RMSE of RH prediction

In [ ]:
# Calculate RMSE
print('RMSE of Decision Tree Regression:')
np.sqrt(mean_squared_error(y_pred, y_test))

#### <u>Conclusion:<u>(Decision Tree Regression)

In [ ]:
# write your conclusion here

### 9) Random Forest Regression<a name="RF"></a>

Let's apply Random Forest regression and measure RMSE

In [ ]:
from sklearn.ensemble import RandomForestRegressor           # Import random forest regressor
from sklearn.model_selection import GridSearchCV       # Import grid search cv

- Fit the RF model and predict

In [ ]:
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

- RMSE of RH prediction

In [ ]:
# Calculate RMSE
print('RMSE of predicted RH in RF model:')
np.sqrt(mean_squared_error(y_pred, y_test))

- Try to improve on baseline RF model: use `GridSearchCV` to search between different hyperparameters and plot the resulting RMSE
    - use different numbers of estimators
    - use cv of 5 or 10
    - use the correct scoring function
    - then, use the best model hyperparameters to predict on the test data

In [ ]:
grid_searcher = GridSearchCV(model_rf, {'n_estimators': [10, 15, 100]}, cv=5, scoring='neg_mean_squared_error')

In [ ]:
model_rf2 = grid_searcher.fit(X_train, y_train)

In [ ]:
y_pred = model_rf2.best_estimator_.predict(X_test)

In [ ]:
print('RMSE using RF grid search method')  
np.sqrt(mean_squared_error(y_pred, y_test))

- Write here your conclusions regarding the Grid Search method. Did the performance improve? How much?

In [ ]:
# your answer here

### 9.1) Plot box plots of the error <a name="bxplot"></a>
- Plot the box plots of absolute errors at different pridiction range (prediction: <20; 20-40; >40)

In [ ]:
error = abs(y_test - y_pred)
df_errors = pd.DataFrame(data={'error': error, 'target': y_test, 'prediction': y_pred})
data = [
    df_errors[df_errors['prediction'] < 20]['error'].to_numpy(),
    df_errors[(df_errors['prediction'] >= 20) & (df_errors['prediction'] < 40)]['error'].to_numpy(),
    df_errors[df_errors['prediction'] >= 40]['error'].to_numpy(),
]
sns.boxplot(data=data, showfliers=False)
plt.show()

#### <u>Conclusion: Random Forest

In [ ]:
# write conclusion here

### Conclusion<a name="conc"></a>

 - Summarize here your conclusions regarding the models used 
